In [1]:
import orjson

from conda_oci_utils import (
    encode_conda_dist_to_oci_dist,
    is_valid_oci_dist,
    is_valid_conda_dist,
)
from conda_forge_metadata.repodata import fetch_repodata, all_labels

In [2]:
import sys
import tqdm

labels = all_labels(use_remote_cache=True)

max_name_len = 0
max_tag_len = 0
for label in tqdm.tqdm(labels, ncols=80, file=sys.stderr):
    for subdir in [
        "linux-64",
        "osx-64",
        "noarch",
        "win-64",
        "linux-aarch64",
        "osx-arm64",
        "linux-ppc64le",
    ]:
        rd = fetch_repodata([subdir], label=label)[0]
        with open(rd) as f:
            rd = orjson.loads(f.read())

        for key in ["packages", "packages.conda"]:
            for dist in rd[key]:
                if label != "main":
                    conda_dist = f"conda-forge/label/{label}/{subdir}/{dist}"
                else:
                    conda_dist = f"conda-forge/{subdir}/{dist}"

                if not is_valid_conda_dist(conda_dist):
                    tqdm.tqdm.write(f"> invalid conda dist: {conda_dist} (label: {label})")

                oci_dist = encode_conda_dist_to_oci_dist(conda_dist)

                if not is_valid_oci_dist(oci_dist):
                    tqdm.tqdm.write(f"> invalid   oci dist: {oci_dist} (label: {label})")
                else:
                    name, tag = oci_dist.rsplit(":", maxsplit=1)
                    # if len(name) > max_name_len:
                    #     tqdm.tqdm.write(f"current maximum oci name length: {len(name)}")
                    max_name_len = max(max_name_len, len(name))
                    # if len(tag) > max_tag_len:
                    #     tqdm.tqdm.write(f"current maximum oci tag length: {len(tag)}")
                    max_tag_len = max(max_tag_len, len(tag))
                    # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")

print(f"max oci name length: {max_name_len}")
print(f"max oci tag length: {max_tag_len}")

  7%|██▉                                      | 26/360 [00:00<00:02, 151.01it/s]

> invalid   oci dist: conda-forge/label/TEST/linux-64/cgdal:2.3.3-py27hf242f0b__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/linux-64/cgdal:2.3.3-py36hf242f0b__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/linux-64/cgdal:2.3.3-py37hf242f0b__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/linux-64/clibgdal:2.3.3-hdb8f723__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/osx-64/cgdal:2.3.3-py27h7eb7563__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/osx-64/cgdal:2.3.3-py36h7eb7563__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/osx-64/cgdal:2.3.3-py37h7eb7563__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/osx-64/clibgdal:2.3.3-h16bfee5__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/win-64/cgdal:2.3.3-py36hdf5ee75__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/win-64/cgdal:2.3.3-py37hdf5ee75__1 (label: TEST)
> invalid   oci dist: conda-forge/label/TEST/win-64/cl

  7%|██▉                                      | 26/360 [00:01<00:02, 151.01it/s]

> invalid conda dist: conda-forge/label/cf201901/osx-64/lammps-2018.03.16-.tar.bz2 (label: cf201901)


  7%|██▉                                      | 26/360 [00:03<00:02, 151.01it/s]

> invalid conda dist: conda-forge/label/cf202003/osx-64/lammps-2018.03.16-.tar.bz2 (label: cf202003)


100%|█████████████████████████████████████████| 360/360 [00:23<00:00, 15.10it/s]

max oci name length: 93
max oci tag length: 81


In [3]:
import defaults_repodata

for channel in defaults_repodata.DEFAULTS_CHANNELS:
    labels = ["main", "borken", "borked"]
    # does not work right now
    # labels = defaults_repodata.all_labels(channel=channel)
    # print(channel, labels)

    max_name_len = 0
    max_tag_len = 0
    for label in tqdm.tqdm(labels, ncols=80, desc=channel):
        for subdir in [
            "linux-64",
            "osx-64",
            "noarch",
            "win-64",
            "linux-aarch64",
            "osx-arm64",
            "linux-ppc64le",
        ]:
            rd = defaults_repodata.fetch_repodata(channel, [subdir], label=label)[0]
            with open(rd) as f:
                rd = orjson.loads(f.read())

            for key in ["packages", "packages.conda"]:
                for dist in rd[key]:
                    if label != "main":
                        conda_dist = f"{channel}/label/{label}/{subdir}/{dist}"
                    else:
                        conda_dist = f"{channel}/{subdir}/{dist}"

                    if (
                        (not is_valid_conda_dist(conda_dist))
                        and "__anaconda_core_depends" not in conda_dist
                    ):
                        tqdm.tqdm.write(f"> invalid conda dist: {conda_dist} (label: {label})")

                    oci_dist = encode_conda_dist_to_oci_dist(conda_dist)

                    if not is_valid_oci_dist(oci_dist):
                        tqdm.tqdm.write(f"> invalid   oci dist: {oci_dist} (label: {label})")
                    else:
                        name, tag = oci_dist.rsplit(":", maxsplit=1)
                        # if len(name) > max_name_len:
                        #     tqdm.tqdm.write(f"current maximum oci name length: {len(name)}")
                        max_name_len = max(max_name_len, len(name))
                        # if len(tag) > max_tag_len:
                        #     tqdm.tqdm.write(f"current maximum oci tag length: {len(tag)}")
                        max_tag_len = max(max_tag_len, len(tag))
                        # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")

    print(f"max oci name length: {max_name_len}")
    print(f"max oci tag length: {max_tag_len}")

main: 100%|███████████████████████████████████████| 3/3 [00:01<00:00,  1.91it/s]


max oci name length: 64
max oci tag length: 48


msys2: 100%|█████████████████████████████████████| 3/3 [00:00<00:00, 504.28it/s]


max oci name length: 44
max oci tag length: 21


r: 100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 12.68it/s]

max oci name length: 41
max oci tag length: 30
